In [1]:
import pandas as pd
df = pd.read_csv("C:\\Users\\martinX1\\Downloads\\Ranking_dataset_v2\\Ranking_dataset_v2.csv", nrows = 10026)
df.head(3)

,listing_id,doc_rank,clicked,n0,n1,n2,n3,n4,n5,n6,...,n40,n41,n42,n43,n44,c0,c1,c2,c3,c4
0,0,0,0,87.0,86.0,87.0,0.00000,0.142328,0.007693,0.0,...,99999.0,0.048142,-0.131414,-0.125540,19.99,0,0,4,0,0
1,0,1,0,87.0,86.0,87.0,0.02381,0.758929,0.091036,0.0,...,1000.0,0.060197,-0.131414,-0.125540,17.90,0,0,4,0,0
2,0,2,0,87.0,86.0,87.0,0.00000,0.133929,0.001270,0.0,...,5.0,-0.119726,-0.276582,-0.213422,9.75,0,0,3,0,0


In [2]:
df.tail(3)

,listing_id,doc_rank,clicked,n0,n1,n2,n3,n4,n5,n6,...,n40,n41,n42,n43,n44,c0,c1,c2,c3,c4
10023,205,57,0,0.0,0.0,0.0,0.014881,0.586310,0.036480,0.0,...,4000.0,0.793980,0.811956,0.799193,103.00,0,0,4,0,2
10024,205,58,0,0.0,0.0,0.0,0.017857,0.895833,0.057064,0.0,...,700.0,0.910541,0.811956,0.799193,84.99,0,0,4,0,2
10025,205,59,0,0.0,0.0,0.0,0.000000,0.336310,0.043632,0.0,...,365.0,0.819111,0.811956,0.799193,174.00,0,0,4,0,2


In [3]:
import math

train_size = 0.8

train_queries = [i for i in range(math.floor(len(df['listing_id'].unique())*train_size)+1)]
test_queries = [i for i in range(math.floor(len(df['listing_id'].unique())*train_size)+1, len(df['listing_id'].unique()) )]

X_train_num = df[df['listing_id'].isin(train_queries)].iloc[:,3:44]
X_test_num = df[df['listing_id'].isin(test_queries)].iloc[:,3:44]

X_train_cat = df[df['listing_id'].isin(train_queries)].iloc[:,45:]
X_test_cat = df[df['listing_id'].isin(test_queries)].iloc[:,45:]

y_train = df[df['listing_id'].isin(train_queries)]['clicked']
y_test = df[df['listing_id'].isin(test_queries)]['clicked']

qids_train = df[df['listing_id'].isin(train_queries)]['listing_id']
qids_test = df[df['listing_id'].isin(test_queries)]['listing_id']


In [4]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
"""df.iloc[:,3:48]
PCA
if 0 < n_components < 1 and svd_solver == ‘full’, 
select the number of components such that the amount of variance that needs
to be explained is greater than the percentage specified by n_components
"""

pipe_numerical = Pipeline([
        ('scale', StandardScaler())
        ,('pca', PCA(n_components=0.99, svd_solver='full'))
])

X_train_transformed = pipe_numerical.named_steps.pca.fit_transform(X_train_num)
X_test_transformed = pipe_numerical.named_steps.pca.transform(X_test_num)

print(len(pipe_numerical.named_steps.pca.components_ ))

print(pipe_numerical.named_steps.pca.explained_variance_ratio_.cumsum())

4
[0.56890524 0.83539279 0.98149021 0.99969373]


In [5]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

enc_categotical = OneHotEncoder()
x_train = enc_categotical.fit_transform(X_train_cat).toarray() 
X_train_transformed = np.concatenate((X_train_transformed, x_train), axis=1)

x_test = enc_categotical.transform(X_test_cat).toarray() 
X_test_transformed = np.concatenate((X_test_transformed, x_test), axis=1)

In [6]:
#https://github.com/szdr/RankNet
def ndcg(y_true, y_score, k=60):
    y_true = y_true.ravel()
    y_score = y_score.ravel()
    y_true_sorted = sorted(y_true, reverse=True)
    ideal_dcg = 0
    for i in range(k):
        ideal_dcg += (2 ** y_true_sorted[i] - 1.) / np.log2(i + 2)
    dcg = 0
    argsort_indices = np.argsort(y_score)[::-1]
    for i in range(k):
        dcg += (2 ** y_true[argsort_indices[i]] - 1.) / np.log2(i + 2)
    ndcg = dcg / ideal_dcg
    return ndcg

In [7]:
import pyltr 
metric = pyltr.metrics.NDCG(k=60)

#hyperpatameter tuning with some grid search would be good here
model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=200, #1000,
    learning_rate=0.02,
    max_features=0.5,
    query_subsample=0.5,
    max_leaf_nodes=20,
    min_samples_leaf=64,
    verbose=1,
)

model.fit(X_train_transformed, y_train, qids_train)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.2792       0.0326        4.12m                                         
    2       0.2953       0.0074        4.00m                                         
    3       0.3088       0.0086        3.91m                                         
    4       0.3393       0.0081        4.10m                                         
    5       0.3298      -0.0105        4.28m                                         
    6       0.3569       0.0013        4.26m                                         
    7       0.3523      -0.0002        4.26m                                         
    8       0.3817      -0.0056        4.26m                                         
    9       0.3488       0.0025        4.42m                                         
   10       0.3524      -0.0084        4.49m                                         
   15       0.3330       0.0018        4.60m         

In [8]:
train_pred = model.predict(X_train_transformed) 
test_pred = model.predict(X_test_transformed)

print('model train: {}'.format( ndcg(y_train, train_pred)))
print('model test: {}'.format( ndcg(y_test, test_pred) ))

model train: 0.48959769113552865
model test: 0.03256945927011301


In [9]:
import xgboost as xgb
from xgboost import DMatrix

In [10]:
#hyperpatameter tuning with some grid search would be good here
params = {'objective': 'rank:pairwise', 'eta': 0.1, 'gamma': 1.0,
               'min_child_weight': 0.2, 'max_depth': 4}
train_dmatrix = DMatrix(X_train_transformed, y_train)
xgb_model = xgb.train(params, train_dmatrix, num_boost_round=100)

[11:26:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[11:26:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[11:26:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[11:26:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[11:26:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[11:26:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[11:26:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

In [11]:
test_dmatrix = DMatrix(X_test_transformed, y_test)
train_pred = xgb_model.predict(train_dmatrix)
test_pred = xgb_model.predict(test_dmatrix)


In [12]:
print('model train: {}'.format( ndcg(y_train, train_pred)))
print('model test: {}'.format( ndcg(y_test, test_pred) ))

model train: 0.3650399013515364
model test: 0.03430616189018335


In [13]:
#https://github.com/szdr/RankNet
from chainer import Chain
import chainer.functions as F
import chainer.links as L
class MLP(Chain):

    def __init__(self, n_in, n_hidden):
        super(MLP, self).__init__(
            l1=L.Linear(n_in, n_hidden),
            l2=L.Linear(n_hidden, n_hidden),
            l3=L.Linear(n_hidden, 1)
        )

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)


class RankNet(Chain):

    def __init__(self, predictor):
        super(RankNet, self).__init__(predictor=predictor)

    def __call__(self, x_i, x_j, t_i, t_j):
        s_i = self.predictor(x_i)
        s_j = self.predictor(x_j)
        s_diff = s_i - s_j
        if t_i.data > t_j.data:
            S_ij = 1
        elif t_i.data < t_j.data:
            S_ij = -1
        else:
            S_ij = 0
        self.loss = (1 - S_ij) * s_diff / 2. + F.log(1 + F.exp(-s_diff))
        return self.loss

In [14]:
scaler = MinMaxScaler()
Xs_train_transformed = scaler.fit_transform(X_train_transformed)
Xs_test_transformed = scaler.transform(X_test_transformed)

In [15]:
#https://github.com/szdr/RankNet
from chainer import Variable, optimizers
n_iter = 500
n_hidden = 100
loss_step = 100

n_dim = np.shape(X_train_transformed)[1]
N_train = np.shape(Xs_train_transformed)[0]

ys_train = np.reshape(np.array(y_train), (-1, 1))

model = RankNet(MLP(n_dim, n_hidden))
optimizer = optimizers.Adam()
optimizer.setup(model)

for step in range(n_iter):
        i, j = np.random.randint(N_train, size=2)
        x_i = Variable(Xs_train_transformed[i].reshape(1, -1).astype(np.float32))
        x_j = Variable(Xs_train_transformed[j].reshape(1, -1).astype(np.float32))
        y_i = Variable(ys_train[i])
        y_j = Variable(ys_train[j])
        model.zerograds()
        loss = model(x_i, x_j, y_i, y_j)
        loss.backward()
        optimizer.update()
        if (step + 1) % loss_step == 0:
            train_score = model.predictor(Variable(Xs_train_transformed.astype(np.float32)))
            test_score = model.predictor(Variable(Xs_test_transformed.astype(np.float32)))
            train_ndcg = ndcg(y_train, train_score.data)
            test_ndcg = ndcg(y_test, test_score.data)
            print("step: {}".format(step + 1))
            print("NDCG@100 | train: {}, test: {}".format(
                train_ndcg, test_ndcg))

C:\Users\martinX1\Anaconda3\lib\site-packages\chainer\optimizers\adam.py:111: RuntimeWarning: invalid value encountered in sqrt
  param.data -= hp.eta * (self.lr * m / (numpy.sqrt(vhat) + hp.eps) +


step: 100
NDCG@100 | train: 0.015829695578341917, test: 0.04473053902311328
step: 200
NDCG@100 | train: 0.09121974140473357, test: 0.08043342583179233
step: 300
NDCG@100 | train: 0.09630724328366908, test: 0.09186772408866929
step: 400
NDCG@100 | train: 0.1463366860370006, test: 0.04729607574922303
step: 500
NDCG@100 | train: 0.08819112008299165, test: 0.015482083372709088


In [16]:
train_pred = model.predictor(Variable(Xs_train_transformed.astype(np.float32)))
test_pred = model.predictor(Variable(Xs_test_transformed.astype(np.float32)))

print('model train: {}'.format( ndcg(y_train, train_pred.data)))
print('model test: {}'.format( ndcg(y_test, test_pred.data) ))

model train: 0.08819112008299165
model test: 0.015482083372709088
